# Cosmos DB Synapse Sales Processing Spark Notebook

## This Spark/PySpark Notebook demonstrates how to:

- Read the Synapse Link Analytic Datastore with Spark/PySpark in Azure Synapse
- Filter the sales data (by doctype, timestamp) while reading it 
- Aggregating the sales data by customer_id
- Displaying the "shape" of the dataframes, and observed schema
- Writing the aggregated "materialized view" of sales-by-customer to the Cosmos DB views container

### Location

- GitHub repo:  https://github.com/cjoakim/azure-cosmos-db-playground
- File in repo: other/synapse/notebooks/cosmos_sql_sales_processing.ipynb

Chris Joakim, Cosmos DB GBB, Microsoft


# Polulating the sales container

In this GitHub repo, in directory '\apis\nosql\dotnet', run the following command:

```
> python main.py load_sales retail sales sales1.json 99999
```


In [23]:
# Load the SynapseLink Sales Data into a Dataframe.
# Select just the "sale" document types from the sales container, 
# which have a minimum _ts (timestamp) value

from pyspark.sql.functions import col

# initialize variables
begin_timestamp = 0  # 1675711974 
end_timestamp   = 1699999999

# read just the doctype "sales", not "line_item"
# "cosmos.oltp" = CosmosDB live database
# "cosmos.olap" = Synapse Link Analytic Datastore
df_sales = spark.read\
    .format("cosmos.olap")\
    .option("spark.synapse.linkedService", "gbbcjcdbnosql_retail_db")\
    .option("spark.cosmos.container", "sales")\
    .load().filter(col("doctype") == "sale")\
    .filter(col("_ts") > begin_timestamp)\
    .filter(col("_ts") < end_timestamp)

display(df_sales.limit(3))


StatementMeta(sparkpool3m, 0, 23, Finished, Available)

SynapseWidget(Synapse.DataFrame, 99f9ea1d-a503-4d04-8df3-88ea54c327cb)

In [24]:
# Display the shape and observed schema of the DataFrame

print('df_sales, shape: {} x {}'.format(
        df_sales.count(), len(df_sales.columns)))
        
df_sales.printSchema()


StatementMeta(sparkpool3m, 0, 24, Finished, Available)

df_sales, shape: 9069 x 19
root
 |-- _rid: string (nullable = true)
 |-- _ts: long (nullable = true)
 |-- pk: string (nullable = true)
 |-- id: string (nullable = true)
 |-- sale_id: long (nullable = true)
 |-- doctype: string (nullable = true)
 |-- date: string (nullable = true)
 |-- line_num: long (nullable = true)
 |-- customer_id: long (nullable = true)
 |-- store_id: long (nullable = true)
 |-- upc: string (nullable = true)
 |-- price: double (nullable = true)
 |-- qty: long (nullable = true)
 |-- cost: double (nullable = true)
 |-- seq: long (nullable = true)
 |-- _etag: string (nullable = true)
 |-- dow: string (nullable = true)
 |-- item_count: long (nullable = true)
 |-- total_cost: double (nullable = true)



In [25]:
# Aggregate Sales by Customer 

import pyspark.sql.functions as F 

df_customer_aggregated = df_sales.groupBy("customer_id") \
    .agg(
        F.first('id').alias('id'), \
        F.first('customer_id').alias('pk'), \
        F.count("customer_id").alias('order_count'), \
        F.sum("total_cost").alias("total_dollar_amount"), \
        F.sum("item_count").alias("total_item_count")) \
        .sort("customer_id", ascending=True)

display(df_customer_aggregated.limit(10))


StatementMeta(sparkpool3m, 0, 25, Finished, Available)

SynapseWidget(Synapse.DataFrame, 823a6565-f0bc-41fa-abee-f35382386dd4)

In [26]:
# Display the shape and observed schema of the DataFrame

print('df_customer_aggregated, shape: {} x {}'.format(
        df_customer_aggregated.count(), len(df_customer_aggregated.columns)))
        
df_customer_aggregated.printSchema()


StatementMeta(sparkpool3m, 0, 26, Finished, Available)

df_customer_aggregated, shape: 5908 x 6
root
 |-- customer_id: long (nullable = true)
 |-- id: string (nullable = true)
 |-- pk: long (nullable = true)
 |-- order_count: long (nullable = false)
 |-- total_dollar_amount: double (nullable = true)
 |-- total_item_count: long (nullable = true)



In [27]:
# Write the customer-aggregated DataFrame to the CosmosDB
# sales_aggregates container.  The id and pk is the customer ID,
# and upserts are enabled.

df_customer_aggregated.write.format("cosmos.oltp")\
    .option("spark.synapse.linkedService", "gbbcjcdbnosql_retail_db")\
    .option("spark.cosmos.container", "views")\
    .mode('append')\
    .save()


StatementMeta(sparkpool3m, 0, 27, Finished, Available)

# Truncating the views container

In this GitHub repo, in directory '\apis\nosql\dotnet', run the following command:

```
> dotnet run truncate_container retail views
```
